In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [4]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate())
    
    x_prev = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1  = add!(model, constvar(:add_1, 1.0))
    x      = add!(model, randomvar(:x))
    
    x_prev_add = add!(model, make_node(+, x_prev, add_1, x))
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    y     = add!(model, datavar(:y, Float64))
    
    add_x_and_noise = add!(model, make_node(+, x, noise, y))
    
    activate!(model)
    
    return x_prev, x, y
end

function kalman(data)
    N = length(data)
    
    x_prev, x, y = kalman_filter_graph()

    link(x_prev, getmarginal(x))
    
    update!(x_prev, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [5]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [6]:
@time kalman(data); # Initial compilation

  3.203412 seconds (9.27 M allocations: 486.872 MiB, 6.34% gc time)


In [7]:
@time kalman(data); # Subsequent runs

  0.000996 seconds (17.34 k allocations: 859.641 KiB)


In [8]:
@btime kalman($data) # Performance benchmark

  653.465 μs (17342 allocations: 859.64 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=4.98176041926534, σ=14.002800840280099)
 Normal{Float64}(μ=17.32327159365538, σ=9.950371902099892)
 Normal{Float64}(μ=18.15046280893522, σ=8.137884587711596)
 Normal{Float64}(μ=10.990902249054386, σ=7.053456158585984)
 Normal{Float64}(μ=11.527645504840178, σ=6.311944030978033)
 Normal{Float64}(μ=12.428915465162184, σ=5.763904177042351)
 Normal{Float64}(μ=10.409410352859876, σ=5.337605126836238)
 Normal{Float64}(μ=11.461014295857995, σ=4.993761694389224)
 Normal{Float64}(μ=14.696148082277633, σ=4.708816093480111)
 Normal{Float64}(μ=18.42766646747086, σ=4.467670516087703)
 Normal{Float64}(μ=18.693416914116558, σ=4.2601432284230505)
 Normal{Float64}(μ=18.185888831736534, σ=4.079085082240021)
 Normal{Float64}(μ=19.46181336445403, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=588.7929799030131, σ=0.582706653680032)
 Normal{Float64}(μ=589.7598141119809, σ=0.5822126418123303)
 Normal{Float64}(μ=590.7234938296723, σ=0.5817198842703781)
 Nor